In [63]:
%reset -f

# 1- Les bibliothèques à importer

In [64]:
import pandas as pd
import ydata_profiling as yp
from ydata_profiling import ProfileReport
from sklearn.preprocessing import LabelEncoder

# 2- Importation, compréhension, Prétraitement et Transformation des données des bases de données

## 2-1 Importation et compréhension des bases de données

In [65]:
dataset = pd.read_csv('dataset.csv', delimiter=',', quotechar='"')
datasetsupplement = pd.read_csv('dataset_supplement.csv', delimiter=',', quotechar='"')


Il s'agit de deux bases de données liées par la variable id dans la base dataset et movie_id dans la base dataset_supplement. Ces deux bases de données donnent des informations sur 4803 films. un premier constat, certaines variables (exemple des variables actors, production-crew, attributs_spoken_languages, etc.) contiennent une liste de dictionnaires, où chaque dictionnaire représente un unité et contient plusieurs attributs. Il est donc indispensable d'extraire les informations pertinentes des données brutes de ces variables pour les rendre exploitables pour le machine learning

## 2-2 Prétraitement

La première étape est d'identifier ces variables, surtout celle qui sont utile pour le machine learning, ensuite d'identifier les informations à extraire. Les variables concernées sont : 
1 - pour la base dataset : spoken_languages, countries_of_production, production, keywords et genres ; 
2 - pour la base dataset_supplement : actors et production_crew.

Pour la suite, les vafriables production et keywords ne seront pas pris en compte à cause du très grands nombre de modalités

#### Fonction pour extraire les modalités d'un attribut complexe

In [66]:

def extraire_modalites(row, variable, attribut):
    elements = eval(row[variable])
    modalites = set()
    for element in elements:
        if attribut in element:
            modalites.add(element[attribut])
    return modalites

#### Fonction pour compter les occurrences de chaque modalité

In [67]:
def compter_occurrences(row, variable, attribut, modalite):
    elements = eval(row[variable])
    nb_occurrences = sum(1 for element in elements if attribut in element and element[attribut] == modalite)
    return nb_occurrences

#### Fonction pour ajouter les nouvelles variables à la base de données

In [68]:
def ajouter_nouvelles_variables(data, variable, attribut):
    modalites = set()
    for index, row in data.iterrows():
        modalites.update(extraire_modalites(row, variable, attribut))
    for modalite in modalites:
        nom_colonne = variable + '_' + attribut + '_' + str(modalite)
        data[nom_colonne] = data.apply(lambda x: compter_occurrences(x, variable, attribut, modalite), axis=1)
    # Ajouter une variable avec le nombre de modalités par attribut
    Nb_modalite_nom_colonne = 'Nb_modalite' + variable + '_' + attribut
    data[Nb_modalite_nom_colonne] = data.apply(lambda x: len(extraire_modalites(x, variable, attribut)), axis=1)

#### Liste des attributs à considérer par variable

In [69]:
#dans la base dataset
attributs_actors = ['gender']
attributs_production_crew = ['department', 'gender']

# Dans la base supplémentaire
attributs_spoken_languages = ['name']
attributs_countries_of_production = ['name']
attributs_genres = ['name']

#### Extraction des informations et création des variables

In [ ]:
for attribut in attributs_production_crew:
    ajouter_nouvelles_variables(datasetsupplement, 'production_crew', attribut)

for attribut in attributs_actors:
    ajouter_nouvelles_variables(datasetsupplement, 'actors', attribut)

for attribut in attributs_spoken_languages:
    ajouter_nouvelles_variables(dataset, 'spoken_languages', attribut)

for attribut in attributs_countries_of_production:
    ajouter_nouvelles_variables(dataset, 'countries_of_production', attribut)

for attribut in attributs_genres:
    ajouter_nouvelles_variables(dataset, 'genres', attribut)

In [72]:
# Sauvegarder la nouvelle base de données
datasetsupplement.to_csv('datasetsupplement_new.csv', index=False)
dataset.to_csv('dataset_new.csv', index=False)

# 3 Sélection des variables

### Fusion des bases de données

In [81]:
# Fusionner les DataFrames sur les colonnes "id" et "movie_id"
merged_dataset = pd.merge(dataset, datasetsupplement, left_on='id', right_on='movie_id')

# Réorganiser les colonnes pour déplacer la colonne "id" en début de base de données
merged_dataset = merged_dataset[['id'] + [col for col in merged_dataset.columns if col != 'id']]

# Supprimer la colonne "movie_id" qui est devenue redondante après la fusion
merged_dataset.drop(columns=['movie_id', 'Unnamed: 0_x', 'Unnamed: 0_y'], inplace=True)

# Sauvegarder le DataFrame fusionné dans un nouveau fichier CSV
merged_dataset.to_csv('merged_dataset.csv', index=False)


### Suppression des variables

1- les variables suivantes vont êtres supprimées dans la mesures où les informations utiles ont étéextraites : 'production_crew', 'actors', 'spoken_languages', 'genres',
2 - Les variables suivantes sont supprimées parce que trop d'information : 'title_x', 'tagline', 
3 - Les variables créées pour chaque langue ont été supprimées compte tenu du nombre de langue élevé. Seule la variables renseignant chaque le nombre de langue a été maintenu

In [82]:
# Liste des variables à supprimer
variables_a_supprimer = ['production_crew', 'actors', 'spoken_languages', 'genres', 'title_x', 'tagline', 'overview' , 'original_title' , 'homepage' , 'spoken_languages_name_', 'spoken_languages_name_Bosanski', 'spoken_languages_name_普通话', 'spoken_languages_name_广州话 / 廣州話', 'spoken_languages_name_తెలుగు', 'spoken_languages_name_Slovenščina', 'spoken_languages_name_Íslenska', 'spoken_languages_name_বাংলা', 'spoken_languages_name_svenska', 'spoken_languages_name_Srpski', 'spoken_languages_name_Slovenčina', 'spoken_languages_name_Gaeilge', 'spoken_languages_name_Català', 'spoken_languages_name_Latin', 'spoken_languages_name_हिन्दी', 'spoken_languages_name_فارسی', 'spoken_languages_name_Polski', 'spoken_languages_name_Türkçe', 'spoken_languages_name_Bamanankan', 'spoken_languages_name_No Language', 'spoken_languages_name_Português', 'spoken_languages_name_Bahasa indonesia', 'spoken_languages_name_Español', 'spoken_languages_name_Nederlands', 'spoken_languages_name_български език', 'spoken_languages_name_suomi', 'spoken_languages_name_ภาษาไทย', 'spoken_languages_name_العربية', 'spoken_languages_name_Esperanto', 'spoken_languages_name_isiZulu', 'spoken_languages_name_Český', 'spoken_languages_name_日本語', 'spoken_languages_name_اردو', 'spoken_languages_name_қазақ', 'spoken_languages_name_Dansk', 'spoken_languages_name_ქართული', 'spoken_languages_name_한국어/조선말', 'spoken_languages_name_Tiếng Việt', 'spoken_languages_name_Italiano', 'spoken_languages_name_shqip', 'spoken_languages_name_??????', 'spoken_languages_name_Wolof', 'spoken_languages_name_Somali', 'spoken_languages_name_پښتو', 'spoken_languages_name_Français', 'spoken_languages_name_Cymraeg', 'spoken_languages_name_English', 'spoken_languages_name_Română', 'spoken_languages_name_Pусский', 'spoken_languages_name_Eesti', 'spoken_languages_name_Deutsch', 'spoken_languages_name_ਪੰਜਾਬੀ', 'spoken_languages_name_Hrvatski', 'spoken_languages_name_ελληνικά', 'spoken_languages_name_Norsk', 'spoken_languages_name_Afrikaans', 'spoken_languages_name_Kiswahili', 'spoken_languages_name_עִבְרִית', 'spoken_languages_name_Galego', 'spoken_languages_name_தமிழ்', 'spoken_languages_name_Magyar', 'spoken_languages_name_Український', 'countries_of_production_name_Colombia', 'countries_of_production_name_Singapore', 'countries_of_production_name_Pakistan', 'countries_of_production_name_Romania', 'countries_of_production_name_Thailand', 'countries_of_production_name_Czech Republic', 'countries_of_production_name_United States of America', 'countries_of_production_name_Slovakia', 'countries_of_production_name_Sweden', 'countries_of_production_name_Malta', 'countries_of_production_name_United Arab Emirates', 'countries_of_production_name_Bulgaria', 'countries_of_production_name_Poland', 'countries_of_production_name_Kazakhstan', 'countries_of_production_name_Spain', 'countries_of_production_name_India', 'countries_of_production_name_Iceland', 'countries_of_production_name_Ecuador', 'countries_of_production_name_France', 'countries_of_production_name_China', 'countries_of_production_name_Jordan', 'countries_of_production_name_Finland', 'countries_of_production_name_Kyrgyz Republic', 'countries_of_production_name_Japan', 'countries_of_production_name_Aruba', 'countries_of_production_name_Belgium', 'countries_of_production_name_Austria', 'countries_of_production_name_South Africa', 'countries_of_production_name_Hungary', 'countries_of_production_name_Denmark', 'countries_of_production_name_Greece', 'countries_of_production_name_Lithuania', 'countries_of_production_name_Guadaloupe', 'countries_of_production_name_Iran', 'countries_of_production_name_Bolivia', 'countries_of_production_name_Netherlands', 'countries_of_production_name_Cameroon', 'countries_of_production_name_Tunisia', 'countries_of_production_name_Luxembourg', 'countries_of_production_name_Bhutan', 'countries_of_production_name_Dominican Republic', 'countries_of_production_name_Bosnia and Herzegovina', 'countries_of_production_name_Morocco', 'countries_of_production_name_Taiwan', 'countries_of_production_name_New Zealand', 'countries_of_production_name_Nigeria', 'countries_of_production_name_Slovenia', 'countries_of_production_name_Malaysia', 'countries_of_production_name_Monaco', 'countries_of_production_name_Brazil', 'countries_of_production_name_Peru', 'countries_of_production_name_Cambodia', 'countries_of_production_name_Serbia and Montenegro', 'countries_of_production_name_Cyprus', 'countries_of_production_name_Afghanistan', 'countries_of_production_name_Angola', 'countries_of_production_name_Fiji', 'countries_of_production_name_Israel', 'countries_of_production_name_Lebanon', 'countries_of_production_name_Canada', 'countries_of_production_name_Russia', 'countries_of_production_name_Indonesia', 'countries_of_production_name_Bahamas', 'countries_of_production_name_Kenya', 'countries_of_production_name_Italy', 'countries_of_production_name_Germany', 'countries_of_production_name_South Korea', 'countries_of_production_name_Libyan Arab Jamahiriya', 'countries_of_production_name_Norway', 'countries_of_production_name_Ukraine', 'countries_of_production_name_Mexico', 'countries_of_production_name_Serbia', 'countries_of_production_name_Turkey', 'countries_of_production_name_Hong Kong', 'countries_of_production_name_United Kingdom', 'countries_of_production_name_Switzerland', 'countries_of_production_name_Guyana', 'countries_of_production_name_Dominica', 'countries_of_production_name_Jamaica', 'countries_of_production_name_Ireland', 'countries_of_production_name_Australia', 'countries_of_production_name_Argentina', 'countries_of_production_name_Chile', 'countries_of_production_name_Algeria', 'countries_of_production_name_Egypt', 'countries_of_production_name_Portugal', 'countries_of_production_name_Philippines', 'countries_of_production_name_Panama', 'Nb_modaliteproduction_crew_gender', 'Nb_modaliteactors_gender', ]

# Supprimer les variables spécifiées
merged_dataset.drop(columns=variables_a_supprimer, inplace=True)

In [84]:
merged_dataset.to_csv('merged_dataset.csv', index=False)

### Exploratory data analysis

In [86]:
#dataset.describe()
profile = ProfileReport(merged_dataset)
profile.to_widgets()
profile.to_file("rapport.html")

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

/media/ayedesso/9CEA8269EA823F8E/Travail_ubuntu/Autres_travaux/ML_dodo/venvir/lib/python3.10/site-packages/ydata_profiling/model/correlations.py:66: UserWarning: There was an attempt to calculate the auto correlation, but this failed.
To hide this warning, disable the calculation
(using `df.profile_report(correlations={"auto": {"calculate": False}})`
If this is problematic for your use case, please report this as an issue:
https://github.com/ydataai/ydata-profiling/issues
(include the error message: 'could not convert string to float: 'en'')
  warnings.warn(
/media/ayedesso/9CEA8269EA823F8E/Travail_ubuntu/Autres_travaux/ML_dodo/venvir/lib/python3.10/site-packages/seaborn/matrix.py:260: FutureWarning: Format strings passed to MaskedConstant are ignored, but in future may error or produce different behavior
  annotation = ("{:" + self.fmt + "}").format(val)
/media/ayedesso/9CEA8269EA823F8E/Travail_ubuntu/Autres_travaux/ML_dodo/venvir/lib/python3.10/site-packages/ydata_profiling/model/mis

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Analyse du rapport

Les données présentent une qualité globalement satisfaisante, avec l'absence de valeurs en double et un faible pourcentage de cellules manquantes (0.4%). La diversité des types de variables, comprenant des variables numériques, catégoriques, de type DateTime et de type texte, suggère la nécessité d'utiliser une gamme variée de techniques d'analyse et de modélisation pour explorer et comprendre les données. Avec 4803 observations, l'ensemble de données est de taille moyenne à grande, offrant suffisamment de données pour mener des analyses significatives et construire des modèles prédictifs robustes. 

Les alertes fournies par le profil de données indiquent plusieurs aspects à considérer :

    Déséquilibre des variables catégorielles : Plusieurs variables présentent un déséquilibre important dans leurs modalités, ce qui peut biaiser les résultats des analyses et des modèles construits. Elles seront supprimées. 

    Données manquantes : Certaines variables contiennent un pourcentage non négligeable de valeurs manquantes. 

    Zéros dans les variables numériques : Plusieurs variables numériques présentent un pourcentage de zéros non négligeable. notemment les variables liées aux différents département de production. Certaines ont été supprimées. Par contre d'autre ont été maintenue du fait de la valeur ajoutée quelle apporte au film. C'est le cas par exemple de la variable 'production_crew_department_Visual Effects'

In [89]:
# Liste des variables à supprimer
variables_a_supprimer2 = ['status', 'original_language', 'genres_name_Documentary', 'genres_name_Animation', 'genres_name_Western' , 'genres_name_Foreign' , 'genres_name_War' , 'genres_name_History' , 'genres_name_Music' , 'genres_name_TV Movie' , 'production_crew_department_Actors', 'vote_average' , 'financial_investment' , 'production_crew_department_Production' , 'production_crew_department_Camera' , 'production_crew_department_Sound' , 'production_crew_department_Writing' , 'production_crew_department_Editing' , 'production_crew_department_Costume & Make-Up' , 'production_crew_department_Crew']

# Supprimer les variables spécifiées
merged_dataset.drop(columns=variables_a_supprimer2, inplace=True)

# Supprimer les individus ayant la valeur 0 pour le revenu
merged_dataset = merged_dataset[merged_dataset['revenue'] != 0]

# Réinitialiser l'index après la suppression des lignes
merged_dataset.reset_index(drop=True, inplace=True)

In [90]:
#dataset.describe()
profile = ProfileReport(merged_dataset)
profile.to_widgets()
profile.to_file("rapport2.html")

/media/ayedesso/9CEA8269EA823F8E/Travail_ubuntu/Autres_travaux/ML_dodo/venvir/lib/python3.10/site-packages/ydata_profiling/utils/dataframe.py:137: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={"index": "df_index"}, inplace=True)


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render widgets:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

### Analyse du deuxième rapport

Après modification apportée à la base, cette étape consistera à analyser les corrélation entre la variables. Pour éviter la colinéarité, les variables indépendantes corrélées entre elles ont été analysées de près. Sur la base de la revu de litérrature, certaines ont été supprimées. Aussi, celle qui sont correlées avec la variables revenue sont supprimées. 

In [91]:
# Liste des variables à supprimer
variables_a_supprimer3 = ['Nb_modaliteproduction_crew_department', 'production_crew_department_Art' , 'production_crew_gender_0' ,	'production_crew_gender_1'	, 'production_crew_gender_2', 'vote_count' , 'popularity']

# Supprimer les variables spécifiées
merged_dataset.drop(columns=variables_a_supprimer3, inplace=True)

/tmp/ipykernel_41854/774090869.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  merged_dataset.drop(columns=variables_a_supprimer3, inplace=True)


In [92]:
merged_dataset.describe()

,id,duration,revenue,Nb_modalitespoken_languages_name,Nb_modalitecountries_of_production_name,genres_name_Thriller,genres_name_Comedy,genres_name_Romance,genres_name_Mystery,genres_name_Crime,...,genres_name_Adventure,genres_name_Action,genres_name_Family,Nb_modalitegenres_name,production_crew_department_Directing,production_crew_department_Lighting,production_crew_department_Visual Effects,actors_gender_0,actors_gender_1,actors_gender_2
count,3395.000000,3395.000000,3.347000e+03,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,...,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000,3395.000000
mean,45688.181738,110.329897,1.169427e+08,1.513991,1.391753,0.284242,0.348159,0.178792,0.079234,0.159352,...,0.199116,0.277761,0.112518,2.634757,1.873638,0.663034,2.086892,8.150221,5.691605,11.737555
std,74871.359795,21.086254,1.836818e+08,0.963703,0.796088,0.451119,0.476457,0.383235,0.270144,0.366058,...,0.399395,0.447961,0.316050,1.118640,1.510988,1.685016,5.622828,12.544974,5.876005,8.310793
min,5.000000,0.000000,5.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,5698.500000,96.000000,1.527744e+07,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,2.000000,1.000000,0.000000,0.000000,2.000000,3.000000,6.000000
50%,11615.000000,106.000000,5.137686e+07,1.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,3.000000,1.000000,0.000000,0.000000,4.000000,4.000000,10.000000
75%,47711.500000,120.500000,1.400367e+08,2.000000,2.000000,1.000000,1.000000,0.000000,0.000000,0.000000,...,0.000000,1.000000,0.000000,3.000000,2.000000,0.000000,2.000000,9.000000,7.000000,15.000000
max,417859.000000,338.000000,2.787965e+09,9.000000,12.000000,1.000000,1.000000,1.000000,1.000000,1.000000,...,1.000000,1.000000,1.000000,7.000000,22.000000,34.000000,104.000000,164.000000,107.000000,88.000000
